In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
%matplotlib inline
import os
import gc
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
import seaborn as sns

In [ ]:
data = pd.read_csv('drive/MyDrive/Colab Notebooks/hand-gesture-recognition-2nd-edition/train.csv')
test = pd.read_csv('drive/MyDrive/Colab Notebooks/hand-gesture-recognition-2nd-edition/test.csv')
ss = pd.read_csv('drive/MyDrive/Colab Notebooks/hand-gesture-recognition-2nd-edition/sample_submission.csv')

In [ ]:
data.head()

,id,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,...,pixel57141,pixel57142,pixel57143,pixel57144,pixel57145,pixel57146,pixel57147,pixel57148,pixel57149,pixel57150,pixel57151,pixel57152,pixel57153,pixel57154,pixel57155,pixel57156,pixel57157,pixel57158,pixel57159,pixel57160,pixel57161,pixel57162,pixel57163,pixel57164,pixel57165,pixel57166,pixel57167,pixel57168,pixel57169,pixel57170,pixel57171,pixel57172,pixel57173,pixel57174,pixel57175,pixel57176,pixel57177,pixel57178,pixel57179,pixel57180
0,0,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
Xdata =data.iloc[:,range(2,57183)]
ydata =data['label']
Xdata.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel57141,pixel57142,pixel57143,pixel57144,pixel57145,pixel57146,pixel57147,pixel57148,pixel57149,pixel57150,pixel57151,pixel57152,pixel57153,pixel57154,pixel57155,pixel57156,pixel57157,pixel57158,pixel57159,pixel57160,pixel57161,pixel57162,pixel57163,pixel57164,pixel57165,pixel57166,pixel57167,pixel57168,pixel57169,pixel57170,pixel57171,pixel57172,pixel57173,pixel57174,pixel57175,pixel57176,pixel57177,pixel57178,pixel57179,pixel57180
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
data[['label','id']].groupby(by= 'label').count()

,id
label,
1,190
2,183
3,183
4,186
5,185
6,194
7,194
8,184
9,182


In [ ]:
Xdata = Xdata // 255

In [ ]:
Xdata['pixel1000'].unique()

array([0, 1])

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain , Xval , ytrain , yval = train_test_split( Xdata.values , ydata.values , test_size=0.2, random_state=42)
#Scaler = StandardScaler()
#Scaler.fit(Xtrain)
#Xtrain = Scaler.transform(Xtrain)
#Xval = Scaler.transform(Xval)

In [ ]:
from sklearn.svm import LinearSVC 
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensemble import GradientBoostingClassifier
#from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier

In [ ]:

#'RandomForestClassifier' : RandomForestClassifier(),#'LinearSVC' :  LinearSVC() ,
models={ 
         'SVC' : SVC(max_iter=100 ) ,
         'KNN' : KNeighborsClassifier(),    
        ##'GradientBoosting':GradientBoostingClassifier(),
        ##'XGBoost' : XGBClassifier(),
        #'LightGBM' : LGBMClassifier()
        'RandomForestClassifier' : RandomForestClassifier(),
        'LinearSVC' :  LinearSVC() 

        }

results= {}
for model_name,model in models.items():
  model.fit(Xtrain , ytrain)
  results[model_name]=model.score(Xval, yval)

results_df=pd.DataFrame(results.values(),results.keys(), columns=['accuracy'])
results_df

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


,accuracy
SVC,0.996667
KNN,0.995000
RandomForestClassifier,0.993333
LinearSVC,0.991667


In [ ]:
print('aaa')

aaa


In [ ]:
model = SVC(decision_function_shape='ovo', max_iter=1000 )
#model.fit(Xtrain , ytrain)
model.fit(Xdata , ydata)

SVC(decision_function_shape='ovo', max_iter=1000)

In [ ]:
model.fit(Xval, yval)

SVC(decision_function_shape='ovo', max_iter=1000)

In [ ]:
model.score(Xval,yval)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  "X does not have valid feature names, but"


1.0

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
np.random.seed(42)
log_reg_grid = { 'C' : np.logspace(0.01 , 1.01 , 10) , 
                 'kernel' : ['linear','poly','rbf','sigmoid'],
                'gamma' : ['scale', 'auto'], 
                'max_iter' : [100,200,300] ,
                'decision_function_shape' : ['ovr', 'ovo']}
rs_log_reg = RandomizedSearchCV(estimator=SVC() , param_distributions=log_reg_grid , cv=5 , n_iter=5 , verbose=1 )
rs_log_reg.fit(Xtrain , ytrain)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).

RandomizedSearchCV(cv=5, estimator=SVC(), n_iter=5,
                   param_distributions={'C': array([ 1.02329299,  1.32163372,  1.70695559,  2.20461792,  2.84737354,
        3.67752434,  4.74970533,  6.13448032,  7.92298601, 10.23292992]),
                                        'decision_function_shape': ['ovr',
                                                                    'ovo'],
                                        'gamma': ['scale', 'auto'],
                                        'kernel': ['linear', 'poly', 'rbf',
                                                   'sigmoid'],
                                        'max_iter': [100, 200, 300]},
                   verbose=1)

In [ ]:
print(rs_log_reg.best_params_)
rs_log_reg.score(Xval, yval)

{'max_iter': 200, 'kernel': 'linear', 'gamma': 'scale', 'decision_function_shape': 'ovo', 'C': 1.3216337213922862}


0.9983333333333333

In [ ]:
testdata = test.drop(columns = ['id'])
testdata.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel57141,pixel57142,pixel57143,pixel57144,pixel57145,pixel57146,pixel57147,pixel57148,pixel57149,pixel57150,pixel57151,pixel57152,pixel57153,pixel57154,pixel57155,pixel57156,pixel57157,pixel57158,pixel57159,pixel57160,pixel57161,pixel57162,pixel57163,pixel57164,pixel57165,pixel57166,pixel57167,pixel57168,pixel57169,pixel57170,pixel57171,pixel57172,pixel57173,pixel57174,pixel57175,pixel57176,pixel57177,pixel57178,pixel57179,pixel57180
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
testdata = testdata // 255

In [ ]:
pred_labels = model.predict(testdata.values)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  "X does not have valid feature names, but"


In [ ]:
pred_labels[:10]

array([14, 10, 14,  5,  7, 12,  1, 15,  2,  3])

In [ ]:
#check simple_submission 
ss.head()

,id,label
0,3000,0
1,3001,0
2,3002,0
3,3003,0
4,3004,0


In [ ]:
ss['label'] = pred_labels
ss.head()

,id,label
0,3000,14
1,3001,10
2,3002,14
3,3003,5
4,3004,7


In [ ]:
ss.to_csv('Sub5.csv', index=False)